tb - 7/5/2022 - Analyze the output of Jerry's first online runs comparing specific humidity and relative humidity across climates

# Imports

In [1]:
import numpy as np
import xarray as xr

# Paths

In [8]:
path_folder = '/DFS-L/DATA/pritchard/tbeucler/For_Jerry/June2022_CI_Runs/'
Rescalings = ['sp','specificNN','relativeNN']
Climates = ['m4k','0k','p4k']

# Load all runs as `xarray` datasets

In [25]:
SP_Ref = {}; RHruns = {}; Qruns = {};

In [29]:
for clim in Climates:
    print(clim)
    SP_Ref[clim] = xr.open_mfdataset(path_folder+Rescalings[0]+'_'+clim+'/A*h1*.nc',
                             combine='nested',concat_dim='time')
    RHruns[clim] = {}; Qruns[clim] = {};
    
    for idropout in range(4):
        RHruns[clim][idropout] = {}; 
        Qruns[clim][idropout] = {}
        
        for BN in ['False','True']:
            
            path_RH = path_folder+Rescalings[2]+'_'+clim+\
            '/relhum_version_'+str(idropout)+'_'+BN
            path_Q = path_folder+Rescalings[1]+'_'+clim+\
            '/spehum_version_'+str(idropout)+'_'+BN
            
            RHruns[clim][idropout][BN] = xr.open_mfdataset(path_RH+'/A*h1*.nc',
                                                           combine='nested',concat_dim='time')
            Qruns[clim][idropout][BN] = xr.open_mfdataset(path_RH+'/A*h1*.nc',
                                                          combine='nested',concat_dim='time')

m4k
0k
p4k


In [28]:
path_RH+'/A*h1*.nc'

'/DFS-L/DATA/pritchard/tbeucler/For_Jerry/June2022_CI_Runs/sp_m4k/relhum_version_0_False/A*h1*.nc'

In [34]:
RHruns['m4k'][0]['False']

,Array,Chunk
Bytes,2.83 kB,16 B
Shape,"(177, 2)","(1, 2)"
Count,531 Tasks,177 Chunks
Type,object,numpy.ndarray
,Array,Chunk
Bytes,1.42 kB,8 B
Shape,"(177,)","(1,)"
Count,531 Tasks,177 Chunks
Type,|S8,numpy.ndarray
,Array,Chunk
